In [20]:
import queue
import threading
import os
import requests
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup

In [45]:
def mygobuster(turl):
    #target = "http://172.17.0.2:1337/"
    dirfound = open("directories.txt", "a")
    htmlfound = open("html.txt", "a")
    global wordlist
    wordlist = [a.strip() for a in open("wordlist.txt")]
    filters = [".html",".css","",".js", ".jpg",".md",".png"]
    for word in wordlist:
        for f in filters:
            url = turl+word+f
            re = requests.head(url)
            
            if re.status_code==301:
                print("Directory found: ", re.url, re.status_code)
                dirfound.write(re.url+"\n")
                newurl = re.url+"/"
                mygobuster(newurl) 
            
            elif re.status_code==200 and ".html" in re.url:
                print("Html file found: ", re.url, re.status_code)
                htmlfound.write(re.url+"\n")
                
            elif re.status_code==200 and "css" in re.url:
                print("Css file found: ", re.url, re.status_code)
            
            elif re.status_code==200 and "js" in re.url:
                print("Javascript found: ", re.url)
            
            elif re.status_code==200:
                print(re.url)
                
                


In [46]:
def threadbuster():
    threads = len(wordlist)
    for i in range(threads):
        t = threading.Thread(target=mygobuster)
        t.start()

In [47]:
r  = requests.get(" http://172.17.0.2:1337/data/about/index.html")
data = r.text
soup = BeautifulSoup(data)

for link in soup.find_all('a'):
    print(link.get("href"))


In [48]:
import scrapy

In [49]:
mygobuster("http://172.17.0.2:1337/")

Directory found:  http://172.17.0.2:1337/data 301
Directory found:  http://172.17.0.2:1337/data/about 301
Html file found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/faq 301
Html file found:  http://172.17.0.2:1337/data/faq/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Html file found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/data/datenschutz 301
Html file found:  http://172.17.0.2:1337/data/datenschutz/index.html 200
Directory found:  http://172.17.0.2:1337/data/about 301
Html file found:  http://172.17.0.2:1337/data/about/index.html 200
Directory found:  http://172.17.0.2:1337/data/impressum 301
Html file found:  http://172.17.0.2:1337/data/impressum/index.html 200
Directory found:  http://172.17.0.2:1337/img 301
Directory found:  http://172.17.0.2:1337/img/background-images 301
http://172.17.0.2:1337/img/background-images/pic1.jpg
http://172.17.0.2:

In [50]:
cat directories.txt

http://172.17.0.2:1337/data/about
http://172.17.0.2:1337/data/faq
http://172.17.0.2:1337/data/impressum
http://172.17.0.2:1337/data/datenschutz
http://172.17.0.2:1337/data/about
http://172.17.0.2:1337/data/impressum
http://172.17.0.2:1337/img/background-images
http://172.17.0.2:1337/static/bootstrap-4.0.0/js
http://172.17.0.2:1337/static/js
http://172.17.0.2:1337/static/css
http://172.17.0.2:1337/static/bootstrap-4.0.0
http://172.17.0.2:1337/data
http://172.17.0.2:1337/img
http://172.17.0.2:1337/js
http://172.17.0.2:1337/static
http://172.17.0.2:1337/favicon_io


In [51]:
cat html.txt

http://172.17.0.2:1337/data/about/index.html
http://172.17.0.2:1337/data/faq/index.html
http://172.17.0.2:1337/data/impressum/index.html
http://172.17.0.2:1337/data/datenschutz/index.html
http://172.17.0.2:1337/data/about/index.html
http://172.17.0.2:1337/data/impressum/index.html
http://172.17.0.2:1337/static/bootstrap-4.0.0/index.html
http://172.17.0.2:1337/about.html
http://172.17.0.2:1337/index.html
http://172.17.0.2:1337/biete.html
http://172.17.0.2:1337/angebote.html
http://172.17.0.2:1337/faq.html
http://172.17.0.2:1337/impressum.html
http://172.17.0.2:1337/datenschutz.html
http://172.17.0.2:1337/angebote.html
http://172.17.0.2:1337/myangebote.html
http://172.17.0.2:1337/suche.html
http://172.17.0.2:1337/about.html
http://172.17.0.2:1337/impressum.html
